<div align="center">
<h2 style="color:#a70000; font-weight:bold">How do we deploy these ML Model now?</h2>
<h3 style="color:#a70000; font-weight:bold">Python has a solution to it: Serialization!</h3>
</div>

> In computer science, in the context of data storage, serialization is the process of translating data structures or object state into a format that can be stored (for example, in a file or memory buffer, or transmitted across a network connection link) and reconstructed later in the same or another computer environment.

In Python, pickling is a standard way to store objects and retrieve them as their original state. To give a simple example:

In [2]:
list_to_pickle = [1, 'here', 123, 'walker']

#Pickling the list
import dill as pickle

with open('./list.pk', 'wb') as filename:
    pickle.dump(list_to_pickle, filename)

In [3]:
!ls *.pk

list.pk


### Contents of the pickle file are nothing but all your data stored in bytes!

When we load the pickle back:

In [4]:
with open('./list.pk', 'rb') as filename:
    loaded_pickle = pickle.load(filename)
    
print("Content of the pickle: {}".format(loaded_pickle))

Content of the pickle: [1, 'here', 123, 'walker']


### For folks who are more familiar with R

$$.rda\;==\;.pk$$

<div align="center">
<h2 style="color:#a70000; font-weight:bold">For our hack session we already have a model, pickled and ready</h2>


<img src="http://download.gamezone.com/uploads/image/data/1220233/article_post_width_pick.JPG" height="500px" width="500px"/>
<div>

## How to load our pickled model

In [5]:
import dill as pickle
clf = 'model_v1.pk'

In [6]:
loaded_model = None
with open('../working_api/working_api/models/'+clf, 'rb') as f:
    loaded_model = pickle.load(f)

### Test dataset:

In [7]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv('../working_api/test_data/test1.csv', encoding="utf-8-sig")
df = df.head()

In [9]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [10]:
loaded_model.predict(df)

array([1, 1, 1, 1, 1])

<div align="center">
<h2 style="color:#a70000; font-weight:bold">Structure of Hack Session</h2>
</div>

<div class="divider_h1" align="center">
<h1 style="font-weight:bold;"><font color="#5ea774">DEMO: HELLO-WORLD<font></h1>
</div>

- Importing the __Flask__ class

In [11]:
from flask import Flask

- Creating an instance of the class:

In [12]:
app = Flask(__name__)

- Since we are dealing with Web APIs and endpoints, __Flask__ saves us the hassle by providing decorators, `route()` to automatically trigger the `url` that we mention:

In [43]:
#@app.route('/')

- __What is a decorator?__

    * In layman's term, decorators are __Funceptions__. Functions within functions.
    * Example:

In [14]:
def divide(a, b):
    return(a/b)

In [15]:
divide(3, 4)

0.75

In [16]:
divide(3, 0)

ZeroDivisionError: division by zero

- Create a decorator to check for the same:

In [54]:
def avoid_error(func):
    def inner(a, b): 
        print("Dividing {} by {}".format(a, b))
        if b == 0:
            print("Not possible to perform division")
            return("Check for your denom")
        else:
            return(a/b)
    return inner

In [55]:
@avoid_error
# Passed on to inner function
def divide(a, b):
    return(a,b)

In [56]:
divide(1,2)

Dividing 1 by 2


0.5

In [57]:
divide(1,0)

Dividing 1 by 0
Not possible to perform division


'Check for your denom'

In [58]:
@app.route('/')
def hello_world():
    return("Hello, world!")

- Content in `return()` in the above definition will be displayed in the browser.

<div class="divider_h1" align="center">
<h1 style="font-weight:bold;"><font color="#5ea774">DEMO:WRAPPING ML MODELS<font></h1>
</div>

### Client Side code:

In [60]:
import json
import requests

- #### Setting the headers to send and accept json responses

In [61]:
header = {'Content-Type': 'application/json', \
                  'Accept': 'application/json'}

- #### Reading test batch

In [63]:
df = pd.read_csv('../working_api/test_data/test1.csv', encoding="utf-8-sig")
df = df.head()
print(df)

    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001015   Male     Yes          0      Graduate            No   
1  LP001022   Male     Yes          1      Graduate            No   
2  LP001031   Male     Yes          2      Graduate            No   
3  LP001035   Male     Yes          2      Graduate            No   
4  LP001051   Male      No          0  Not Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5720                  0       110.0             360.0   
1             3076               1500       126.0             360.0   
2             5000               1800       208.0             360.0   
3             2340               2546       100.0             360.0   
4             3276                  0        78.0             360.0   

   Credit_History Property_Area  
0             1.0         Urban  
1             1.0         Urban  
2             1.0         Urban  
3             NaN     

- #### Converting Pandas Dataframe to json

In [65]:
data = df.to_json(orient='records')

In [66]:
data

'[{"Loan_ID":"LP001015","Gender":"Male","Married":"Yes","Dependents":"0","Education":"Graduate","Self_Employed":"No","ApplicantIncome":5720,"CoapplicantIncome":0,"LoanAmount":110.0,"Loan_Amount_Term":360.0,"Credit_History":1.0,"Property_Area":"Urban"},{"Loan_ID":"LP001022","Gender":"Male","Married":"Yes","Dependents":"1","Education":"Graduate","Self_Employed":"No","ApplicantIncome":3076,"CoapplicantIncome":1500,"LoanAmount":126.0,"Loan_Amount_Term":360.0,"Credit_History":1.0,"Property_Area":"Urban"},{"Loan_ID":"LP001031","Gender":"Male","Married":"Yes","Dependents":"2","Education":"Graduate","Self_Employed":"No","ApplicantIncome":5000,"CoapplicantIncome":1800,"LoanAmount":208.0,"Loan_Amount_Term":360.0,"Credit_History":1.0,"Property_Area":"Urban"},{"Loan_ID":"LP001035","Gender":"Male","Married":"Yes","Dependents":"2","Education":"Graduate","Self_Employed":"No","ApplicantIncome":2340,"CoapplicantIncome":2546,"LoanAmount":100.0,"Loan_Amount_Term":360.0,"Credit_History":null,"Property_Are

- #### We'll be sending resources to the server-side, hence __POST__

```python
resp = requests.post("http://<url/ip>/predict", \
                    data = json.dumps(data),\
                    headers= header)
```

### Server Side code:

In [68]:
import pandas as pd

```python
test_json = response.get_json() 
#response object in Flask that contains our dataframe sent our via POST

test = pd.read_json(test_json, orient='records')
```

- #### Converting the json back to pandas dataframe

In [71]:
test = pd.read_json(data, orient='records')

In [76]:
test

,ApplicantIncome,CoapplicantIncome,Credit_History,Dependents,Education,Gender,LoanAmount,Loan_Amount_Term,Loan_ID,Married,Property_Area,Self_Employed
0,5720,0,1.0,0,Graduate,Male,110,360,LP001015,Yes,Urban,No
1,3076,1500,1.0,1,Graduate,Male,126,360,LP001022,Yes,Urban,No
2,5000,1800,1.0,2,Graduate,Male,208,360,LP001031,Yes,Urban,No
3,2340,2546,NaN,2,Graduate,Male,100,360,LP001035,Yes,Urban,No
4,3276,0,1.0,0,Not Graduate,Male,78,360,LP001051,No,Urban,No


- Minor pre-processing steps before we proceed:

In [77]:
#To resolve the issue of TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'
test['Dependents'] = [str(x) for x in list(test['Dependents'])]

#Getting the Loan_IDs separated out
loan_ids = test['Loan_ID']

- <p>Loading our pickled estimator and do predictions:</p>

In [73]:
import dill as pickle
clf = 'model_v1.pk'

In [74]:
loaded_model = None
with open('../working_api/working_api/models/'+clf, 'rb') as f:
    loaded_model = pickle.load(f)

In [78]:
predictions = loaded_model.predict(test)

In [79]:
predictions

array([1, 1, 1, 1, 1])

- <p><font color="#810000" style="font-weight:bold;">jsonify</font> our predictions and send the response back with status code: <font color="#810000" >200</font></p>

In [80]:
prediction_series = list(pd.Series(predictions))
final_predictions = pd.DataFrame(list(zip(loan_ids, prediction_series)))
print(final_predictions)

          0  1
0  LP001015  1
1  LP001022  1
2  LP001031  1
3  LP001035  1
4  LP001051  1


In [81]:
from flask import jsonify

```python
responses = jsonify(predictions=final_predictions.to_json(orient="records"))
responses.status_code = 200
```

<div align="center">
<h2 style="color:#a70000; font-weight:bold;">Demo Run for API: Local</h2>
</div>

In [ ]:
import json
import requests
import pandas as pd

"""Setting the headers to send and accept json responses
"""
header = {'Content-Type': 'application/json', \
                  'Accept': 'application/json'}

"""Reading test batch
"""
df = pd.read_csv('../test_data/test1.csv', encoding="utf-8-sig")
df = df.head()

"""Converting Pandas Dataframe to json

MORE INFO:
==============
orient: string

    Series
    =====================
    default is 'index'
    allowed values are: {'split', 'records', 'index'}
    
    Dataframe
    =====================
    default is 'columns'
    allowed values are: {'split', 'records', 'columns', 'values'}
    
"""
data = df.to_json(orient='records')

"""POST <url>/predict, sending data to the Flask Server

NOTE: 
==================================
- <url> for local: localhost or 127.0.0.1 or 0.0.0.0
- <url> for cloud: DNS or ip of VM created
"""
resp = requests.post("http://0.0.0.0:5000/predict", \
                    data = json.dumps(data),\
                    headers= header)

In [ ]:
resp.status_code

In [ ]:
resp.json()

<div align="center">
<h2 style="color:#a70000; font-weight:bold;">Demo Run for API: Cloud</h2>
</div>

In [ ]:
import json
import requests
import pandas as pd

"""Setting the headers to send and accept json responses
"""
header = {'Content-Type': 'application/json', \
                  'Accept': 'application/json'}

"""Reading test batch
"""
df = pd.read_csv('../test_data/test1.csv', encoding="utf-8-sig")
df = df.head()

"""Converting Pandas Dataframe to json

MORE INFO:
==============
orient: string

    Series
    =====================
    default is 'index'
    allowed values are: {'split', 'records', 'index'}
    
    Dataframe
    =====================
    default is 'columns'
    allowed values are: {'split', 'records', 'columns', 'values'}
    
"""
data = df.to_json(orient='records')

"""POST <url>/predict, sending data to the Flask Server

NOTE: 
==================================
- <url> for local: localhost or 127.0.0.1
- <url> for cloud: DNS or ip of VM created
"""
resp = requests.post("http://<url>:5000/predict", \
                    data = json.dumps(data),\
                    headers= header)

In [ ]:
resp.status_code

In [ ]:
resp.json()